# Assignment No 2
## Paper Number 7
## Title: Title here
text here

------------------------------
Code to make initialization easier, can be ignored since requirements.txt is not handed in.

In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


# Import Dependancies
This part of the code imports necessary Python libraries for the implementation of the Naming Game model:

- Mesa: An agent-based modeling framework that provides a structure for implementing the model
- NumPy: For numerical operations and array handling
- Pandas: For data manipulation and analysis
- Seaborn: For statistical data visualization
- Matplotlib: For creating plots and graphs
- Random: For generating random values and selections
- Scipy: Installed as part of Mesa and used later to fit certain distributions to histogram bins.

In [2]:
#!/usr/bin/env python3.11
!python --version

import mesa
print(f"mesa version: {mesa.__version__}")
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random

import networkx as nx

DEBUGSTEP = True

Python 3.12.10
mesa version: 3.2.0


Create random Graph

In [ ]:
# code for random graph

Create Agent

In [16]:
from mesa import Agent

# code for agents
class GraphAgent(Agent):
    def __init__(self, model, starting_state, possible_states):
        super().__init__(model)
        self.state = starting_state
        self.next_state = starting_state
        self.possible_states = possible_states
        self.theta = 0.5 #TODO make function to get random value here or in initialization

    def step(self):
        # get neighbors and if a certain percentage is above a chosen threshold change next state
        neighbors = self.model.grid.get_neighbors(self.pos, include_center=False) #TODO check if it is detecting the neigbours correctly and not only giving back 0
        if not len(neighbors):
            return
        for state in self.possible_states:
            changed_state_amount = len([neighbor for neighbor in neighbors if neighbor.state == state])
            percentage_changed = changed_state_amount / len(neighbors)
            if percentage_changed > self.theta:
                self.next_state = state
                break
            else:
                self.next_state = self.state
    
    def advance(self):
        self.state = self.next_state

Create Model

In [8]:
from mesa.space import NetworkGrid
from mesa import Model

# code for model
class GraphModel(Model):
    def __init__(self, num_nodes=20, avg_degree=2, starting_state=0, possible_states=[0,1], seed=None):
        super().__init__(seed=seed)
        self.num_agents = num_nodes

        # Generate a sparse random graph using NetworkX
        p_uniform = avg_degree / (num_nodes - 1)
        self.G = nx.fast_gnp_random_graph(num_nodes, p_uniform, 42)
        self.grid = NetworkGrid(self.G)

        # Create an agent for each node
        GraphAgent.create_agents(model=self, n=num_nodes, starting_state=starting_state, possible_states=possible_states)
        for i, agent in enumerate(self.agents):
            self.grid.place_agent(agent, i)

    def step(self):
        self.agents.do("step")
        self.agents.do("advance")

Run Model

In [5]:
def run_model(N, D, t, starting_state, possible_states, iteration):
    print(f"Starting model iteration: {iteration}")
    model = GraphModel(N, D, starting_state, possible_states)
    for i in range(t):
        if i % int(t/10) == 0 and DEBUGSTEP:
            print(f"iter: {iteration}, step: {i}")
            
        model.step()

    #model_data = model.datacollector.get_model_vars_dataframe()
    #agent_data = model.agent_datacollector.get_agent_vars_dataframe()
    #return model_data, agent_data

In [17]:
N = 1000
D = 2
t = 100

STARTING_STATE = 0
POSSIBLE_STATES = [0,1]

run_model(N, D, t, STARTING_STATE, POSSIBLE_STATES, 1)

Starting model iteration: 1
iter: 1, step: 0
iter: 1, step: 10
iter: 1, step: 20
iter: 1, step: 30
iter: 1, step: 40
iter: 1, step: 50
iter: 1, step: 60
iter: 1, step: 70
iter: 1, step: 80
iter: 1, step: 90
